## Actividad 6

In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('temps.csv')
df.sample()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
115,2016,4,28,Thurs,60,61,61.0,64,56,65,62,73


#### 1.  Para identificar anomalías, obtenga un resumen estadístico, o dicho de otro modo, realice un análisis descriptivo de sus datos, comente.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            348 non-null    int64  
 1   month           348 non-null    int64  
 2   day             348 non-null    int64  
 3   week            348 non-null    object 
 4   temp_2          348 non-null    int64  
 5   temp_1          348 non-null    int64  
 6   average         348 non-null    float64
 7   actual          348 non-null    int64  
 8   forecast_noaa   348 non-null    int64  
 9   forecast_acc    348 non-null    int64  
 10  forecast_under  348 non-null    int64  
 11  friend          348 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 32.8+ KB


In [9]:
df.dtypes

year                int64
month               int64
day                 int64
week               object
temp_2              int64
temp_1              int64
average           float64
actual              int64
forecast_noaa       int64
forecast_acc        int64
forecast_under      int64
friend              int64
dtype: object

#### 2.  Realice  un  análisis  gráfico  de  las  temperaturas máximas  históricas, las temperaturas de los dos días anteriores y la estimación del amigo, con respecto a la fecha. Comente si hay anomalías.

#### 3.  Recodifique los días de la semana, creando siete variables 1-0 asociados a los días de la semana. Le puede facilitar el proceso usando get_dummies de la librería Pandas.

#### 4.  Ahora, separe los datos en predictores y variable respuesta. Su variable Y  es  la  temperatura  máxima  real  y  los  predictores  son  todas  las columnas que usa el modelo para hacer una predicción. Asegúrese de transformar  en  una  matriz  Numpy  porque  esa  es  la  forma  en  que funciona el algoritmo.

#### 5.  Separe  su  base  en  entrenamiento  y  testeo  en  75/25.  Use  una  semilla (random_state = 42) para obtener los mismos valores.

#### 6.  Cree una línea de base, es decir, el error que obtendremos si simplemente predijéramos la temperatura máxima promedio para todos los días.

# Ajustar el modelo

#### 7.  Ajuste  un  modelo  de  regresión  de  bosque  aleatorio  con  los  datos  de entrenamiento. Implemente 1000 árboles y use la misma semilla declarada en el punto 5.

#### 8.  Obtenga la predicción. Coméntalo.

#### 9.  Obtenga las métricas de precisión.

#### 10.  ¿Considera que el modelo es adecuado?